Extract monthly climate variables (ERA5Land) for the rgi subregions

In [1]:
import ee
import numpy as np
import json
import pandas as pd
import geopandas as gpd
import os
import numpy as np

In [2]:
#%%
# # # Trigger the authentication flow.
# ee.Authenticate()

# # # Initialize the library.
ee.Initialize()

In [3]:
# define folder and file paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")
folder_climate = os.path.join(folder_AGVA, "Climate")

In [4]:
### get convex hull of the glacier in each O3Region

# load rgi outlines
asset_rgi = ee.FeatureCollection('projects/lzeller/assets/rgi_2km_o3regions')

# aggregate list of unique subregions
subregions = asset_rgi.aggregate_array("O3Region").distinct().sort().remove(0)

# define function that gets convex hull of glaciers
def get_chull(region):
  
    # get geometry of the glaciers
    rgio3 = asset_rgi.filter(ee.Filter.eq('O3Region', region)).geometry()

    # get convex hull of the region
    rgio3_chull = rgio3.convexHull();

    # create feature to return
    feature_return = ee.Feature(rgio3_chull, {"O3Region":region})
    
    return feature_return

# get the bounding geometries for each
o3_chulls = ee.FeatureCollection( subregions.map(lambda x : get_chull(x) ))

# get single bounding chull of those
combined_chull = o3_chulls.geometry().buffer(10000).convexHull()

All CMIP6 models available and usable on GEE:
'ACCESS-CM2', 'ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CESM2', 'CESM2-WACCM', 'CMCC-CM2-SR5', 'CMCC-ESM2', 'CNRM-CM6-1', 'CNRM-ESM2-1', 'CanESM5', 'EC-Earth3', 'EC-Earth3-Veg-LR', 'FGOALS-g3', 'GFDL-CM4', 'GFDL-ESM4', 'GISS-E2-1-G', 'HadGEM3-GC31-LL', 'HadGEM3-GC31-MM', 'IITM-ESM', 'INM-CM4-8', 'INM-CM5-0', 'IPSL-CM6A-LR', 'KACE-1-0-G', 'KIOST-ESM', 'MIROC-ES2L', 'MIROC6', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0', 'NESM3', 'NorESM2-MM', 'TaiESM1', 'UKESM1-0-LL'

'NorESM2-LM' does not have temperature data available for all our years of interest
https://developers.google.com/earth-engine/datasets/catalog/NASA_GDDP-CMIP6

Let's use a 13-member ensemble, as suggested by https://doi.org/10.1002/joc.7566
"ACCESS-ESM1-5", "BCC-CSM2-MR", "CanESM5", "CNRM-ESM2-1", "EC-Earth3", "GFDL-ESM4", "GISS-E2-1-G", "INM-CM5-0", "IPSL-CM6A-LR", "MIROC6", "MPI-ESM1-2-HR", "MRI-ESM2-0", "UKESM1-0-LL"

In [5]:
# first: we will create and save "modern" and "future" winter/summer temperature/precip products (for both scenarios)
# each will have 13 bands: one for each climate model

# set some parameters
years_modern = [2013,2022]
years_future = [2090,2099]
months_s = [6,8]
months_w = [1,3]
variable_t = 'tas'
variable_p = 'pr'


# create a list of all the variable combinations we are going to do
all_years = [ [2013,2022], [2090,2099] ]
all_months = [ [6,9], [1,3] ]
all_vars = ['tas', 'pr']
all_scenarios = ['ssp245', 'ssp585']

all_combos = []
for ys in all_years:
    for ms in all_months:
        for v in all_vars:
            for s in all_scenarios:
                all_combos.append( {'year1':ys[0], 'year2':ys[1], 'month1':ms[0], 'month2':ms[1], 'var':v, 'scenario':s} )

# create ee.list of the 13-members
ensemble_models = ["ACCESS-ESM1-5", "BCC-CSM2-MR", "CanESM5", "CNRM-ESM2-1", "EC-Earth3", "GFDL-ESM4", "GISS-E2-1-G", "INM-CM5-0", "IPSL-CM6A-LR", "MIROC6", "MPI-ESM1-2-HR", "MRI-ESM2-0", "UKESM1-0-LL"]
ensemble_models_fc = ee.FeatureCollection([ee.Feature(None, {"model":m}) for m in ensemble_models])


In [6]:
# now for each of the combinations, go through and create that product and save
for dic in all_combos:
    
    # define output name
    if dic['year1']<2050: era='modern'
    else: era='future'
    if dic['month2']>6: season='summer'
    else: season='winter'
    output_name = f"{era}_{season}_{dic['var']}_{dic['scenario']}"
    print(output_name)
    
    # subset cmip6
    cmip6 = ee.ImageCollection('NASA/GDDP-CMIP6').filter(
                ee.Filter.eq('scenario', dic['scenario'])).filter( 
                ee.Filter.calendarRange(dic['year1'], dic['year2'], 'year')).filter(
                ee.Filter.calendarRange(dic['month1'], dic['month1'], 'month'))
    
    # create function to subset that data to a given model and get the average
    def get_model_mean(feature):

        # subset to these model obs
        cmip6_subset = cmip6.filter(ee.Filter.eq('model', feature.get('model'))).select(dic['var'])

        # get the mean
        this_mean = ee.Image(cmip6_subset.mean()).rename(dic['var'])

        # return the mean
        return this_mean.set({'model':feature.get('model')})

    # then, for each of the ensemble models, get the average
    cmip6_ic = ee.ImageCollection( ensemble_models_fc.map( lambda f : get_model_mean(f) ) )

    # aggregate the model names that are in there
    used_models = cmip6_ic.aggregate_array('model')
    
    # convert IC to multi-band image, clip to AOI, rename bands, add in info
    cmip6_i = cmip6_ic.toBands().clip(combined_chull).rename(used_models).set(
                    {'variable':dic['var'], 'season':season, 'era':era})

    # export to asset
    task = ee.batch.Export.image.toAsset(
        image = cmip6_i, #regional_clipped_image,
        region = combined_chull, # region.bounds()
        maxPixels = int(1e13),
        assetId = f'projects/lzeller/assets/CMIP6/{output_name}',
        description = output_name,
        )

    task.start()
    print('Export started')
    print()
    
    

modern_summer_tas_ssp245
Export started

modern_summer_tas_ssp585
Export started

modern_summer_pr_ssp245
Export started

modern_summer_pr_ssp585
Export started

modern_winter_tas_ssp245
Export started

modern_winter_tas_ssp585
Export started

modern_winter_pr_ssp245
Export started

modern_winter_pr_ssp585
Export started

future_summer_tas_ssp245
Export started

future_summer_tas_ssp585
Export started

future_summer_pr_ssp245
Export started

future_summer_pr_ssp585
Export started

future_winter_tas_ssp245
Export started

future_winter_tas_ssp585
Export started

future_winter_pr_ssp245
Export started

future_winter_pr_ssp585
Export started



In [15]:
### now for each of the 16 subregions, aggregate the modern/future winter/summer p/t in each scenario

# for each subregion, send it off to get the average winter/summer precip/temp
for region_n in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]:
    
    print(f"Starting Region {region_n}")

    # get just this region geometry
    subset_chull = o3_chulls.filter(ee.Filter.eq('O3Region', region_n)).first()
    
    # list to hold data
    all_data = []

    for dic in all_combos:

        # define output name
        if dic['year1']<2050: era='modern'
        else: era='future'
        if dic['month2']>6: season='summer'
        else: season='winter'
        output_name = f"{era}_{season}_{dic['var']}_{dic['scenario']}"
        print(output_name)

        # grab this data product
        data = ee.Image(f'projects/lzeller/assets/CMIP6/{output_name}')
        
        # calculate mean of all the bands (each band is an ensemble member)
        data = data.reduce('mean')
        
        # calculate areal mean within this region
        mean = data.reduceRegion(
                        reducer=ee.Reducer.mean(),
                        geometry=subset_chull.geometry(),
                        scale=1000,  # meters
                    ).get('mean')

        # format into dic
        data_dic = {"O3Region":region_n, 'long_name':output_name,
                    'scenario':dic['scenario'], 'era':era, 'season':season,
                    'variable':dic['var'], 'values':mean.getInfo()
                   }

        # add to list
        all_data.append(data_dic)
        
    # format into df
    all_df = pd.DataFrame(all_data)

    # save to computer
    out_path = os.path.join(folder_climate, "O3Regions", 'CMIP6', f'Region_{str(region_n).zfill(2)}_cmip6.csv')
    all_df.to_csv(out_path, index=False)

    print("This region is done!")
    print()


Starting Region 1
modern_summer_tas_ssp245
modern_summer_tas_ssp585
modern_summer_pr_ssp245
modern_summer_pr_ssp585
modern_winter_tas_ssp245
modern_winter_tas_ssp585
modern_winter_pr_ssp245
modern_winter_pr_ssp585
future_summer_tas_ssp245
future_summer_tas_ssp585
future_summer_pr_ssp245
future_summer_pr_ssp585
future_winter_tas_ssp245
future_winter_tas_ssp585
future_winter_pr_ssp245
future_winter_pr_ssp585
This region is done!

Starting Region 2
modern_summer_tas_ssp245
modern_summer_tas_ssp585
modern_summer_pr_ssp245
modern_summer_pr_ssp585
modern_winter_tas_ssp245
modern_winter_tas_ssp585
modern_winter_pr_ssp245
modern_winter_pr_ssp585
future_summer_tas_ssp245
future_summer_tas_ssp585
future_summer_pr_ssp245
future_summer_pr_ssp585
future_winter_tas_ssp245
future_winter_tas_ssp585
future_winter_pr_ssp245
future_winter_pr_ssp585
This region is done!

Starting Region 3
modern_summer_tas_ssp245
modern_summer_tas_ssp585
modern_summer_pr_ssp245
modern_summer_pr_ssp585
modern_winter_tas_ss